## Query OpenAlex for works authored by a person
This notebook queries the [OpenAlex API](https://docs.openalex.org/api) via its `/works` endpoint for works authored by a person. It takes an ORCID URL from a list of ORCID IDs as input which is used to filter for works where '`authorships.author.orcid`' matches the given ORCID URL.
The notebook iterates through the given list and displays all DOIs and their title.

In [1]:
# Prerequisites:
import requests         # dependency to make HTTP calls

Note: all ORCID IDs must be in the form 'ID' and be seperated from the next with a comma.

In [2]:
list_of_ids=['0000-0001-5380-4449',       
'0000-0001-5406-9458',
'0000-0003-3547-3257',
'0000-0003-3654-5267',
'0000-0003-4331-8695',
'0000-0003-4939-1666',
'0000-0003-4971-9991']

We use it to query the OpenAlex API for works that specified the ORCID URL within their metadata in the field '`authorships.author.orcid`'.
 Since the API uses [pagination](https://docs.openalex.org/api/get-lists-of-entities#pagination), we need to loop through all pages to get the complete result set.

In [3]:
# OpenAlex endpoint to query for works
OPENALEX_API_WORKS = "https://api.openalex.org/works"

# query all works that are connected to orcid
def query_openalex_for_person2works(orcid):
    page = 1
    max_page = 1
    
    while page <= max_page:
        params = {'filter': 'authorships.author.orcid:'+orcid, 'page': page}
        response = requests.get(url=OPENALEX_API_WORKS,
                                params=params,
                                headers= {'Accept': 'application/json'})
        response.raise_for_status()
        result=response.json()

        # calculate max page number in first loop
        if max_page == 1:
            max_page = determine_max_page(result)
        page = page + 1
        yield result

# calculate max number of result pages
def determine_max_page(response_data):
    item_count = response_data['meta']['count']
    items_per_page = response_data['meta']['per_page']
    max_page_ceil = item_count // items_per_page + bool(item_count % items_per_page)
    return max_page_ceil

From the resulting list of works we extract and print out title and DOI. 

*Note: works that do not have a DOI assigned, will not be printed.*

In [4]:
# from the result pages we get from the OpenAlex API, extract the data about works
def extract_works_from_page(page):
    return [work for work in page.get('results') or []]

# extract DOI from work
def extract_doi(work):
    doi=work.get('ids', {}).get('doi') or ""
    doi_id=doi.replace("https://doi.org/", "") if doi else doi
    title=work.get('display_name', "")
    return doi_id, title

def main_search(orcid):
    list_of_pages=query_openalex_for_person2works(orcid)
    for page in list_of_pages or []:
        works=extract_works_from_page(page)
        for work in works or []:
            doi,title=extract_doi(work)
            if doi:
                print(f"{doi},{title}")
                
# main programm:
for item in list_of_ids:
    main_search(item)                

10.1111/cod.12432,European Society of Contact Dermatitis guideline for diagnostic patch testing - recommendations on best practice
10.1111/jdv.13854,A systematic review of worldwide epidemiology of psoriasis
10.1152/ajpcell.1994.266.1.c1,Biological applications of atomic force microscopy
10.1103/physrevb.61.r2389,Tunable two-dimensional photonic crystals using liquid crystal infiltration
10.1007/s11661-006-0134-7,An overall aspect of electroless Ni-P depositions—A review article
10.1128/mcb.15.3.1786,Regulation of cell-type-specific interleukin-2 receptor alpha-chain gene expression: potential role of physical interactions between Elf-1, HMG-I(Y), and NF-kappa B family proteins
10.1161/01.cir.68.5.891,Closed mitral valvotomy: early results and long-term follow-up of 3724 consecutive patients.
10.1080/09540121003623693,Psychosocial challenges and protective influences for socio-emotional coping of HIV+ adolescents in South Africa: a qualitative investigation
10.1111/j.1365-2133.2008.087

10.1111/srt.12016,International guidelines for the <i>in vivo</i> assessment of skin properties in non‐clinical settings: part 1. pH
10.1111/bjd.13169,Association between tobacco smoking and prognosis of occupational hand eczema: a prospective cohort study
10.1093/rheumatology/kem145,Tumour necrosis factor- single nucleotide polymorphisms are not independent of HLA class I in UK Caucasians with adult onset idiopathic inflammatory myopathies
10.1186/1743-7075-9-74,The impact of cow's milk-mediated mTORC1-signaling in the initiation and progression of prostate cancer
10.1111/jdv.14316,Occupational skin diseases: actual state analysis of patient management pathways in 28 European countries
10.1007/s11882-009-0024-3,Acetaminophen use: A risk for asthma?
10.1016/j.compstruct.2006.04.052,Fracture behaviour of composite maritime T-joints
10.1007/s00403-009-0980-4,Short- and long-term effects of single and repetitive glove occlusion on the epidermal barrier
10.1002/14651858.cd004414.pub3,Inter

10.1007/s13312-013-0021-4,50 years of tuberculosis control in India: Progress, pitfalls and the way forward
10.1111/jdv.13602,Non-melanoma skin cancer by solar UV: the neglected occupational threat
10.5414/dbp58034,Bewertung von Epikutantestreaktionen auf âProblemallergeneâ mit vermehrt fraglichen oder schwach positiven Reaktionen
10.1007/3-540-31301-x_15,Clinical Aspects of Irritant Contact Dermatitis
10.1007/s00105-008-1707-8,Neue Formen der Zusammenarbeit zwischen Praxis und Klinik
10.1016/j.jallcom.2009.06.178,Electroless nickel deposition from methane sulfonate bath
10.1016/j.eurger.2011.05.011,Irritability of the skin barrier: A comparison of chronologically aged and photo-aged skin in elderly and young adults
10.1111/j.1463-1318.2007.01210.x,Low anastomotic leak rate after colorectal surgery: a single-centre study
10.1109/16.861582,Design, fabrication, and analysis of SiGeC heterojunction PMOSFETs
10.1109/iedm.1996.553580,Novel SiGeC channel heterojunction PMOSFET
10.1007/s0

10.1002/1348-9585.12083,Outdoor workers’ perceptions of skin cancer risk and attitudes to sun‐protective measures: A qualitative study
10.1111/jdv.17011,Improved protection of outdoor workers from solar ultraviolet radiation: position statement
10.1111/j.1610-0387.2009.07194.x,Zertifizierung: âBerufsdermatologie (ABD)â: Neues Seminar-Curriculum 2010 der Arbeitsgemeinschaft fÃ¼r Berufs- und Umweltdermatologie in der Deutschen Dermatologischen Gesellschaft
10.1007/s00105-003-0644-9,Das neue Hautarztverfahren
10.5414/dbp51002,Begutachtungsempfehlungen für die Berufskrankheit Nr. 5101 der Anlage zur BKV
10.1055/s-0034-1370272,Versorgung des akuten Nierenversagens in Deutschland – Eine strukturelle Analyse
10.1016/j.jphotobiol.2020.111866,Personal ultraviolet radiation dosimetry and its relationship with environmental data: A longitudinal pilot study in Croatian construction workers
10.1111/ddg.12617,S1 guideline on occupational skin products: protective creams, skin cleansers, skin car

10.1136/thx.38.4.279,Results of surgical treatment of ventricular septal defects with pulmonary hypertension.
10.1063/1.121028,Electrical characteristics of plasma oxidized Si1−x−yGexCy metal–oxide–semiconductor capacitors
10.1063/1.123762,Strained Si n-channel metal–oxide–semiconductor transistor on relaxed Si1−xGex formed by ion implantation of Ge
10.1034/j.1600-0536.2002.460413.x,Ammonium thiolactate and thiolactic acid: important hairdressers' allergens?
10.1111/srt.12113,Standardization of skin cleansing<i>in vivo</i>: part II. Validation of a newly developed Automated Cleansing Device (ACiD)
10.1055/s-0034-1391089,„Ich kann mit Handschuhen nicht arbeiten!“ – oder doch? Herausforderungen und Lösungen bei der Auswahl eines adäquaten Handschuhschutzes
10.1017/s2045796014000079,Community mental health teams in low- and middle-income countries
10.1046/j.1439-0353.2004.04028.x,Forum
10.1002/jbmr.3893,Association of Mineral Bone Disorder With Decline in Residual Kidney Function in Incid

...

10.1080/09548980701671094,Human eye-head co-ordination in natural exploration
10.1016/s0969-2126(01)00688-8,Solution Structure of a Telomeric DNA Complex of Human TRF1
10.3389/fnhum.2014.00071,Kinesthetic and vestibular information modulate alpha activity during spatial navigation: a mobile EEG study
10.1167/14.2.20,Spatial biases in viewing behavior
10.1016/j.visres.2005.08.032,Differences of monkey and human overt attention under natural conditions
10.5709/acp-0209-2,Is Attentional Resource Allocation Across Sensory Modalities Task-Dependent?
10.1053/j.gastro.2013.07.015,Mitochondrial Gene Polymorphisms That Protect Mice From Colitis
10.1111/j.0953-816x.2003.03122.x,Stimulus locking and feature selectivity prevail in complementary frequency ranges of V1 local field potentials
10.1023/a:1013776130161,None
10.3389/fnhum.2016.00188,Proposing Metrics for Benchmarking Novel EEG Technologies Towards Real-World Measurements
10.1002/cne.20399,Most peptide-containing sensory neurons lack prot

10.1152/jn.1997.77.1.491,Spatiotemporal Distribution of Intracellular Calcium Transients During Epileptiform Activity in Guinea Pig Hippocampal Slices
10.1007/s00422-006-0050-3,Symbols as Self-emergent Entities in an Optimization Process of Feature Extraction and Predictions
10.1002/ppul.1950070406,Effects of acute viral respiratory tract infections in patients with cystic fibrosis
10.3389/fnbeh.2018.00031,Autonomous Vehicles Require Socio-Political Acceptance—An Empirical and Philosophical Perspective on the Problem of Moral Decision Making
10.1016/j.visres.2009.03.011,Effects of luminance contrast and its modifications on fixation behavior during free viewing of images from different categories
10.1371/journal.pone.0055201,Visualization of Intrapulmonary Lymph Vessels in Healthy and Inflamed Murine Lung Using CD90/Thy-1 as a Marker
10.1371/journal.pone.0166647,Learning New Sensorimotor Contingencies: Effects of Long-Term Use of Sensory Augmentation on the Brain and Conscious Percepti

10.1016/s0925-4927(99)00051-7,Preliminary findings of simultaneous 18F-FDG and 99mTc-HMPAO SPECT in patients with depressive disorders at rest: differential correlates with ratings of anxiety
10.1007/s00441-005-0001-7,Down-regulation of vasoactive intestinal peptide and altered expression of its receptors in rat diabetic cardiomyopathy
10.1093/cercor/bht318,Primary Visual Cortex Represents the Difference Between Past and Present
10.1136/thorax.56.4.331,Sodium cromoglycate in childhood asthma
10.7554/elife.21761,Humans treat unreliable filled-in percepts as more real than veridical ones
10.1007/s00391-017-1347-7,Using multimedia information and communication technology (ICT) to provide added value to reminiscence therapy for people with dementia
10.1007/978-3-540-77343-6_26,Auditory Gist Perception: An Alternative to Attentional Selection of Auditory Streams?
10.1103/physreve.74.041404,Depletion forces between nonspherical objects
10.1088/0954-898x_4_3_001,Alternating oscillatory and st

10.1016/j.neuroimage.2021.118179,Spike-timing-dependent plasticity can account for connectivity aftereffects of dual-site transcranial alternating current stimulation
10.3791/4036,A Human Fallopian Tube Model for Investigation of <em>C. trachomatis</em> Infections
10.1016/s0925-4927(03)00077-5,Simultaneous pattern of rCBF and rCMRGlu in continuation ECT: case reports
10.1007/bf02704159,The depletion potential in one, two and three dimensions
10.1002/cber.19841170234,Kinetische Untersuchungen zu Diels‐Alder‐Reaktionen des Cyclooctins mit anschließender Aromatisierung
10.3389/fnbeh.2016.00085,STN-DBS Reduces Saccadic Hypometria but Not Visuospatial Bias in Parkinson's Disease Patients
10.1002/sdtp.11836,77-1: Invited Paper : Ultra-low Power OLED Microdisplay for Extended Battery Life in NTE Displays
10.1016/j.jid.2018.03.1499,Neutrophil Adhesion Is a Prerequisite for Antibody-Mediated Proteolytic Tissue Damage in Experimental Models of Epidermolysis Bullosa Acquisita
10.1167/13.14.6,Diss

KeyboardInterrupt: 